In [ ]:
!pip install timm albumentations wandb

In [ ]:
import torch
import torch.nn as nn
import timm
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import cv2
from PIL import Image
from torch.cuda.amp import autocast, GradScaler
import wandb
from pathlib import Path

class Config:
    # Model settings
    model_name = 'convnext_large_in22k'
    image_size = 384
    batch_size = 32
    num_epochs = 30
    num_folds = 5
    
    # Training settings
    learning_rate = 1e-4
    min_lr = 1e-6
    weight_decay = 0.01
    
    # Simplified augmentation settings
    train_aug = A.Compose([
        A.Resize(384, 384),
        A.HorizontalFlip(),
        A.RandomBrightnessContrast(),
        A.Normalize(),
        ToTensorV2()
    ])
    
    val_aug = A.Compose([
        A.Resize(384, 384),
        A.Normalize(),
        ToTensorV2()
    ])

class CardDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = cv2.imread(row['filename'])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed['image']
            
        return image, row['label']

class GradingModel(nn.Module):
    def __init__(self, model_name, num_classes):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=True)
        
        # Replace classifier
        in_features = self.model.get_classifier().in_features
        self.model.reset_classifier(0)
        
        self.classifier = nn.Sequential(
            nn.LayerNorm(in_features),
            nn.Dropout(0.5),
            nn.Linear(in_features, 512),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        x = self.model.forward_features(x)
        if len(x.shape) == 3:
            x = x[:, 0]  # Take CLS token for ViT-like models
        x = self.classifier(x)
        return x

def train_fold(fold, train_df, val_df, config, num_classes):
    train_dataset = CardDataset(train_df, config.train_aug)
    val_dataset = CardDataset(val_df, config.val_aug)
    
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, 
                            shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, 
                          shuffle=False, num_workers=4)
    
    model = GradingModel(config.model_name, num_classes=num_classes)
    model = model.cuda()
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), 
                                lr=config.learning_rate,
                                weight_decay=config.weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                                          T_max=config.num_epochs,
                                                          eta_min=config.min_lr)
    scaler = GradScaler()
    
    best_acc = 0
    for epoch in range(config.num_epochs):
        # Training
        model.train()
        train_loss = 0
        for images, labels in train_loader:
            images, labels = images.cuda(), labels.cuda()
            
            optimizer.zero_grad()
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item()
            
        # Validation
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.cuda(), labels.cuda()
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        
        val_acc = correct / total
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), f'best_model_fold{fold}.pth')
            
        scheduler.step()
        
        print(f'Fold {fold}, Epoch {epoch+1}/{config.num_epochs}')
        print(f'Train Loss: {train_loss/len(train_loader):.4f}')
        print(f'Val Loss: {val_loss/len(val_loader):.4f}')
        print(f'Val Acc: {val_acc:.4f}')
        
    return best_acc

def main():
    wandb.init(project="card-grading")
    
    config = Config()
    df = pd.read_csv('../scrape/psa_sales_20250222_170248.csv')
    
    # Print class distribution before encoding
    print("Grade distribution:")
    print(df['grade'].value_counts())
    
    # Data preparation
    le = LabelEncoder()
    df['label'] = le.fit_transform(df['grade'])
    num_classes = len(le.classes_)
    
    print(f"\nNumber of classes: {num_classes}")
    print("Class mapping:")
    for i, grade in enumerate(le.classes_):
        print(f"{grade} -> {i}")
    
    # Handle class imbalance warning
    if len(df['grade'].unique()) < config.num_folds:
        config.num_folds = len(df['grade'].unique())
        print(f"\nWarning: Reducing number of folds to {config.num_folds} due to limited classes")
    
    # Stratified K-Fold
    skf = StratifiedKFold(n_splits=config.num_folds, shuffle=True, random_state=42)
    fold_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['label'])):
        print(f"\nTraining Fold {fold + 1}/{config.num_folds}")
        train_df = df.iloc[train_idx].reset_index(drop=True)
        val_df = df.iloc[val_idx].reset_index(drop=True)
        
        # Print split sizes
        print(f"Training set size: {len(train_df)}")
        print(f"Validation set size: {len(val_df)}")
        
        best_acc = train_fold(fold, train_df, val_df, config, num_classes)
        fold_scores.append(best_acc)
        
    print("\nTraining completed!")
    print(f"Cross-validation scores: {fold_scores}")
    print(f"Mean accuracy: {np.mean(fold_scores):.4f}")
    print(f"Std accuracy: {np.std(fold_scores):.4f}")

if __name__ == "__main__":
    main()